In [1]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

import pandas as pd
import numpy as np

import json

In [3]:
def face_mesh_processor(image_path):
    drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
    with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5) as face_mesh:
        image = cv2.imread(image_path)
            # Convert the RGB image to RGB before processing.
        results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    return results

In [4]:
def landmarks_to_dataframe_processor(results):
    string = str(results.multi_face_landmarks[0])
    string = string.replace('\nlandmark ', '').replace('\n  ', '').replace('\n  ', '').replace('x: ', '').replace('y: ', ', ')
    string = string.replace('z: ', ', ').replace('\n', '').replace('landmark', '').replace(' ', '')
    string = string.split('{')[1:]
    string = [item.replace('}', '') for item in string]
    landmark_list = []
    for item in string:
        sub_list = item.split(',')
        sub_list = [float(x) for x in sub_list]
        landmark_list.append(sub_list)
    return landmark_list

In [5]:
class Face:
    def __init__(self, image_path):
        #reading image
        self.image = cv2.imread(image_path)
        #image dimensions
        self.height = self.image.shape[0]
        self.width = self.image.shape[1]
        #creating a facemesh object with ready .multi_face_landmarks attribute with all landmarks
        self.face_mesh_object = face_mesh_processor(image_path)
        #converting the landmark fromat to a list
        self.landmark_list = landmarks_to_dataframe_processor(self.face_mesh_object)
        #converting the list to a dataframe
        self.dataframe = pd.DataFrame(self.landmark_list).rename(columns = {0:'x', 1:'y', 2:'z'})
        
        self.dataframe['x'] = self.dataframe['x'] * self.width
        self.dataframe['y'] = self.dataframe['y'] * self.height
        
    def write_landmark(self,landmark_index):
        x = int(self.dataframe.iloc[landmark_index].x)
        y = int(self.dataframe.iloc[landmark_index].y)
        cv2.circle(self.image,(x, y), 3,(0, 255, 0), thickness=1, lineType=cv2.LINE_8 )
        cv2.putText(self.image, str(landmark_index) ,org = (x,y), fontFace = cv2.FONT_HERSHEY_SIMPLEX,
                   fontScale = 0.5, color = (255,255,255), lineType = 2, thickness = 1)
        cv2.imshow("image", self.image)
        cv2.waitKey(0)        